In [1]:
# Importing Libraries
import cv2
import numpy as np
import argparse
import time

In [2]:
# Load YOLOv3
net = cv2.dnn.readNet('yolov3.weights','yolov3.cfg.txt')

# Save names of different objects in a list
classes = []
with open('coco.names.txt','r') as f:
    classes = [line.strip() for line in f.readlines()]
        
layer_names = net.getLayerNames()
    
# Return the indices of the output layers of the network
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size = (len(classes), 3))

# Load image
img_path = 'C:/Users/johba/Desktop/Python/OPENCV/office.png'
img = cv2.imread(img_path)
#img = cv2.resize(img, None, fx = .4, fy = .4)
height, width, channels = img.shape

# Detect objects
blob = cv2.dnn.blobFromImage(img, scalefactor=1/255, size=(416,416), 
                             mean=(0,0,0), swapRB=True, crop=False)
net.setInput(blob)
outputs = net.forward(output_layers)

# Box dimensions
boxes = []
confs = []
class_ids = []

for output in outputs:
    for detect in output:
        scores = detect[5:]
        class_id = np.argmax(scores)
        conf = scores[class_id]
        if conf > 0.2:
            # Object detected
            center_x = int(detect[0] * width)
            center_y = int(detect[1] * height)
            w = int(detect[2] * width)
            h = int(detect[3] * height)
            
            # Rectangle coordinates
            x = int(center_x - w/2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confs.append(float(conf))
            class_ids.append(class_id)

# Draw bounding boxes & add labels
indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        cv2.putText(img, label, (x, y - 5), font, 1, color, 1)

cv2.imshow("Image", img)
cv2.waitKey(0)    
cv2.destroyAllWindows()